In [21]:
import ast
import json
import sys
import math
import enum

In [22]:
class Primitives_Estimator:
    def __init__(self):
        pass
    # def estimate_int_size(self, value):        
    #     base_size = sys.getsizeof(0)  
    #     num_digits = math.ceil(num_bits / 30)          
    #     extra_size = (num_digits - 1) * 4
    #     return base_size + extra_size
    # def estimate_str_size(self, value):
    #     base_size = sys.getsizeof('')
    #     extra_size = len(value)
    #     return base_size + extra_size
    # def estimate_float_size(self, value):
    #     base_size = sys.getsizeof(0.0)
    #     return base_size
    # def estimate_bool_size(self, value):
    #     base_size = sys.getsizeof(True)
    #     return base_size
    # def estimate_bytes_size(self, value):
    #     base_size = sys.getsizeof(b'')
    #     extra_size = len(value)
    #     return base_size + extra_size
    # def estimate_byte_array_size(self, value):
    #     base_size = sys.getsizeof(bytearray()) + 1
    #     extra_size = len(value)
    #     return base_size + extra_size
    # def estimate_complex_size(self, value):
    #     base_size = sys.getsizeof(0j)
    #     return base_size
    def estimate_primitive_size(self, value):
        return sys.getsizeof(value)
    def estimate_list_size(self,length):
        def get_precomputed_capacity(length):
            if length == 0:
                return 0
            elif length <= 4:
                return 4
            elif length <= 8:
                return 8
            elif length <= 16:
                return 16
            elif length <= 25:
                return 25
            elif length <= 35:
                return 35
            elif length <= 49:
                return 49
            elif length <= 64:
                return 64
            else:
                return int(length * 1.025)
        base_size = sys.getsizeof([]) + 8 * get_precomputed_capacity(length)
        return base_size
    

In [12]:
class Memory_Parser:
    class AssignTypes(enum.Enum):
        PRIMITIVE = "primitive"
        LIST = "list" 
    def __init__(self):
        self.primitives_estimator = Primitives_Estimator()
        self.vars = {'z' : (10, sys.getsizeof(0), 'int')}
        self.funcs = {'int':('int',0), 'str':('str',0), 'float':('float',0), 'bool':('bool',0), 'bytes':('bytes',0), 'bytearray':('bytearray',0), 'complex':('complex',0)}
        self.primitives=['int','str','float','bool','bytes','bytearray','complex']
    def _hande_primitives_type_conversions(self, node):
        if isinstance(node.func, ast.Name) and node.func.id == 'int':  
                arg_val = self._evaluate_primtive_expression(node.args[0])
                return int(arg_val)
        elif isinstance(node.func, ast.Name) and node.func.id == 'str':
            return str(self._evaluate_primtive_expression(node.args[0]))
        elif isinstance(node.func, ast.Name) and node.func.id == 'float':
            return float(self._evaluate_primtive_expression(node.args[0]))
        return None
    def _handle_mathematical_ops(self, node, left_val, right_val):
        if isinstance(node.op, ast.Add):
            return left_val + right_val
        elif isinstance(node.op, ast.Sub):
            return left_val - right_val
        elif isinstance(node.op, ast.Mult):
            return left_val * right_val
        elif isinstance(node.op, ast.Div):
            return left_val / right_val
        elif isinstance(node.op, ast.Pow):
            return left_val ** right_val
        elif isinstance(node.op, ast.Mod):
            return left_val % right_val
        else:
            raise ValueError(f"Unsupported operation: {type(node.op).__name__}")
        
    def _evaluate_primtive_expression(self, node):
        if isinstance(node, ast.Constant):  
            return node.value
        elif isinstance(node, ast.BinOp):  
            left_val = self._evaluate_primtive_expression(node.left)
            right_val = self. _evaluate_primtive_expression(node.right)
            result = self._handle_mathematical_ops(node, left_val, right_val)
            if result is not None:
                return result    
        elif isinstance(node, ast.Name):  
                if node.id in self.vars:
                    return self.vars[node.id][0]  
                else:
                    raise NameError(f"Variable '{node.id}' is not defined.")
        elif isinstance(node, ast.Call): 
            result = self._hande_primitives_type_conversions(node)
            if result is not None:
                return result
        else:
            raise TypeError(f"Unsupported AST node: {type(node).__name__}")   
           
    def _evaluate_primitive_assignment(self,stmt):
        var_name = stmt.targets[0].id  
        result = self._evaluate_primtive_expression(stmt.value)
        memory = self.primitives_estimator.estimate_primitive_size(result)
        self.vars[var_name] = (result, memory, type(result).__name__) 
    def _assignment_type(self, node):
        if isinstance(node, ast.Constant):  
            if type(node.value).__name__ in self.primitives: 
                return self.AssignTypes.PRIMITIVE
            else:
                return self.AssignTypes.LIST
        elif isinstance(node, ast.List):
            return self.AssignTypes.LIST
        elif isinstance(node, ast.Name):
            if self.vars[node.id][2] in self.primitives:
                return self.AssignTypes.PRIMITIVE
            elif self.vars[node.id][2] == 'list':
                return self.AssignTypes.LIST
        elif isinstance(node, ast.Call):
            if self.funcs[node.func.id][0] in self.primitives:
                return self.AssignTypes.PRIMITIVE
            elif self.funcs[node.func.id][0] == 'list':
                return self.AssignTypes.LIST 
        elif isinstance(node, ast.BinOp):
            left_type = self._assignment_type(node.left)
            right_type = self._assignment_type(node.right)
            if left_type == self.AssignTypes.LIST or right_type == self.AssignTypes.LIST:
                return self.AssignTypes.LIST
            elif left_type == self.AssignTypes.PRIMITIVE and right_type == self.AssignTypes.PRIMITIVE:
                return self.AssignTypes.PRIMITIVE
    def _evaluate_list_assignment(self, stmt):
        def _parse_list_elements_sizes(node):
            if isinstance(node, ast.List):  
                sizes_and_lengths = [_parse_list_elements_sizes(el) for el in node.elts]
                total_size = sum([size for size in sizes_and_lengths])
                total_length = len(node.elts)
                return total_size + self.primitives_estimator.estimate_list_size(total_length)
            elif isinstance(node,ast.Constant):
                return sys.getsizeof(node.value)
            
        var = stmt.targets[0].id
        list_length = len(stmt.value.elts)
        memory = self.primitives_estimator.estimate_list_size(list_length)
        elements_size = _parse_list_elements_sizes(stmt.value)
        self.vars[var] = (list_length,elements_size,'list')
        
          
    def _assigmemt_handler(self,tree):
        stmt = tree.body[0]
        if self._assignment_type(stmt.value) == self.AssignTypes.PRIMITIVE:
            self._evaluate_primitive_assignment(stmt)
        elif self._assignment_type(stmt.value) == self.AssignTypes.LIST:
            print("List Assignment")
            self._evaluate_list_assignment(stmt)
    

In [24]:
x = 'x =[[[]]]'
tree = ast.parse(x)
print(ast.dump(tree, indent=4)) 
print(tree.body[0].value.elts)

Module(
    body=[
        Assign(
            targets=[
                Name(id='x', ctx=Store())],
            value=List(
                elts=[
                    List(
                        elts=[
                            List(elts=[], ctx=Load())],
                        ctx=Load())],
                ctx=Load()))],
    type_ignores=[])


In [26]:
parser = Memory_Parser()
parser._assigmemt_handler(tree)
print(parser.vars)

List Assignment
{'z': (10, 28, 'int'), 'x': (1, 232, 'list')}


In [15]:
print(sys.getsizeof([[],[],[]]))

80


In [16]:
print(parser.primitives_estimator.estimate_list_size(6))

120
